In [1]:
%pip install gym[classic_control]

Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential 
from keras.layers import Dense
from keras.optimizers import Adam
from pygame import gfxdraw
import os

In [3]:
env = gym.make("CartPole-v0", render_mode="rgb_array")
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
batch_size = 32
n_episodes = 1000
output_dir = "model_output/cartpole/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

c:\Users\memed\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [7]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001
        self.model = self._build_model()
 
    def _build_model(self):
        model = Sequential() 
        model.add(Dense(32, activation="relu",
                        input_dim=self.state_size))
        model.add(Dense(32, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss="mse",
                     optimizer=Adam(lr=self.learning_rate))
        return model
 
    def remember(self, state, action, reward, next_state, done): 
        self.memory.append((state, action, reward, next_state, done))

    def train(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward # if done 
        if not done:
            target = (reward + self.gamma * np.amax(self.model.predict(next_state, verbose=0)[0]))
            target_f = self.model.predict(state, verbose=0)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0) 
        if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size) 
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0])

    def save(self, name): 
        self.model.save_weights(name)

In [8]:
agent = DQNAgent(state_size, action_size)

c:\Users\memed\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [9]:
for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state[0], (1, state_size))
    done = False 
    time = 0
    while not done:
        env.render()
        action = agent.act(state)
        next_state, reward, done, _, info = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size]) 
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {}".format(e, n_episodes-1, time, agent.epsilon))
        time += 1
    if len(agent.memory) > batch_size:
        agent.train(batch_size) 
    if e % 50 == 0:
        agent.save(output_dir + "weights_" + "{:04d}".format(e) + ".hdf5")

episode: 0/999, score: 18, e: 1.0
episode: 1/999, score: 15, e: 1.0
episode: 2/999, score: 11, e: 0.99
episode: 3/999, score: 14, e: 0.99
episode: 4/999, score: 17, e: 0.99
episode: 5/999, score: 14, e: 0.98
episode: 6/999, score: 10, e: 0.98
episode: 7/999, score: 24, e: 0.97
episode: 8/999, score: 18, e: 0.97
episode: 9/999, score: 33, e: 0.96
episode: 10/999, score: 15, e: 0.96
episode: 11/999, score: 23, e: 0.95
episode: 12/999, score: 11, e: 0.95
episode: 13/999, score: 14, e: 0.94
episode: 14/999, score: 20, e: 0.94
episode: 15/999, score: 35, e: 0.93
episode: 16/999, score: 11, e: 0.93
episode: 17/999, score: 15, e: 0.92
episode: 18/999, score: 27, e: 0.92
episode: 19/999, score: 12, e: 0.91
episode: 20/999, score: 16, e: 0.91
episode: 21/999, score: 19, e: 0.9
episode: 22/999, score: 19, e: 0.9
episode: 23/999, score: 13, e: 0.9


KeyboardInterrupt: 